In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

c:\Users\gtanu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
file_path = 'interaction_data.csv'
dataset = pd.read_csv(file_path)

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276000 entries, 0 to 275999
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   student_id         276000 non-null  object
 1   video_title        276000 non-null  object
 2   video_link         276000 non-null  object
 3   video_description  276000 non-null  object
 4   video_tags         276000 non-null  object
 5   interaction_type   276000 non-null  object
dtypes: object(6)
memory usage: 12.6+ MB


In [4]:
dataset = dataset.drop_duplicates(subset='video_title', keep='first').reset_index(drop=True)


In [5]:
dataset.head(5)

,student_id,video_title,video_link,video_description,video_tags,interaction_type
0,S1,Manzil 2025: CHEMICAL KINETICS in One Shot: Al...,https://www.youtube.com/watch?v=T0LFfBuifjk,A comprehensive one-shot session on Chemical K...,#ChemicalKinetics #JEEMain #JEEAdvanced #Phys...,liked
1,S1,ElecTroLySiS 01: Class 10 Chemistry ICSE,https://www.youtube.com/watch?v=BEBy1JvNq-A,A detailed explanation of Electrolysis for Cla...,#Electrolysis #Class10 #ICSE #PhysicsWallah #A...,commented
2,S1,MoLE ConCepT in 40 mins : CBSE / ICSE : CHEMIS...,https://www.youtube.com/watch?v=bI-yhj9WgxQ,A 40-minute explanation of the Mole Concept id...,#MoleConcept #CBSE #ICSE #PhysicsWallah #Alakh...,not watched
3,S1,Chemical Bonding 08 | Hybridisation | How to F...,https://www.youtube.com/watch?v=AvhUUY8yD08,This video explains the concept of Hybridisati...,#Hybridisation #ChemicalBonding #IITJEE #NEET ...,not watched
4,S1,Manzil 2025: CHEMICAL BONDING in One Shot: All...,https://www.youtube.com/watch?v=z2-EeL9Qvf4,A one-stop solution to mastering Chemical Bon...,#ChemicalBonding #JEEMain #JEEAdvanced #Manzil...,watched


In [6]:
dataset.tail(5)

,student_id,video_title,video_link,video_description,video_tags,interaction_type
87,S1,Electric Charges and Field | Class 12 Physics ...,https://www.youtube.com/watch?v=MVvHdH6n2VE,Anupam Sir delves into Electric Charges and Fi...,#electriccharges #fields #class12physics #anup...,commented
88,S1,Gravitation | one shot | AP SA1 | ONE SHOT | C...,https://www.youtube.com/watch?v=j9nSxCim93Q,Gayathri Ma’am provides a one-shot explanation...,#gravitation #class9physics #cbse #ap #gayathr...,liked
89,S1,Class 11 Physics NCERT Chapter 4 Vector in One...,https://www.youtube.com/watch?v=Nj10CUCUTwY,This session part of the Dahaad Series provide...,#class11physics #vectors #neet2026 #dahaadseri...,shared
90,S1,Electric Charges & Fields Class 12 (L1) | Clas...,https://www.youtube.com/watch?v=F__Mbfyv7uQ,Anupam Sir covers the fundamentals of Electric...,#electriccharges #fields #class12physics #anup...,not watched
91,S1,Motion in 1D Lecture 1 | Physics NEET 2025 | A...,https://www.youtube.com/watch?v=5GRAW_aAztg,Gaurav Gupta Sir teaches the first lecture on ...,#motionin1d #neet2025 #neetscience #gauravgupt...,liked


In [5]:
dataset['content'] = dataset['video_title'] + dataset['video_description'] + dataset['video_tags']


In [6]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(dataset['content'])

In [7]:
nn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)  # More neighbors to handle duplicates
nn_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [8]:
def recommend_videos(video_title, num_recommendations=5):
    try:
        idx = dataset[dataset['video_title'] == video_title].index[0]
    except IndexError:
        return "Video title not found in the dataset."

    distances, indices = nn_model.kneighbors(tfidf_matrix[idx], n_neighbors=10)  

    similar_indices = indices.flatten()[1:]

    recommended_videos = dataset.iloc[similar_indices][['video_title', 'video_link']].drop_duplicates().head(num_recommendations)

    if recommended_videos.empty:
        return "No sufficient recommendations available."
    
    return recommended_videos.to_dict(orient='records')

In [9]:
example_title = dataset['video_title'].iloc[69]  
recommendations = recommend_videos(example_title, num_recommendations=5)

In [10]:
print("Recommendations:")
for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec['video_title']} - {rec['video_link']}")

Recommendations:
1. Basic Mathematics Class 11 | Lecture 2 | JEE Main | JEE Advanced |Arvind Kalia Sir| Vedantu - https://www.youtube.com/watch?v=eCPAG522Xd4
2. Relations & Functions Full Chapter | Class 11 Maths Chapter 2 | JEE 2025 Maths | Harsh Sir - https://www.youtube.com/watch?v=SWjh91DSVbk
3. Sets Full Chapter | Class 11 Maths Chapter 1 | JEE 2025 Maths | Harsh Sir - https://www.youtube.com/watch?v=1rr6yrqhskg
4. Master Algebra for JEE Main & Advanced - https://www.youtube.com/watch?v=j6NBRQ_FrNE
5. JEE Mains 2025: Coordinate Geometry - Vector 3D Prerequisite- GEOMETRY | JEE Advanced & Main | Ep-2 - https://www.youtube.com/watch?v=HcqMtl0RtaY


In [13]:
!pip install joblib

In [14]:
import joblib
joblib.dump(nn_model, 'nearest_neighbors_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!
